# **CS349F Review Session 2**

# Table of Contents
* [**CS349F Review Session 2**](#**CS349F-Review-Session-2**)
* [Setup Libraries](#Setup-Libraries)
* [Utility Functions](#Utility-Functions)
* [1. Instantiate Your CloudEx Trader Object](#1.-Instantiate-Your-CloudEx-Trader-Object)
	* [1.1 Create Trader Object](#1.1-Create-Trader-Object)
	* [1.2 Get all My Pending Orders](#1.2-Get-all-My-Pending-Orders)
	* [1.3 Get All My Historical Orders](#1.3-Get-All-My-Historical-Orders)
	* [1.4 Get All My Historical Trades](#1.4-Get-All-My-Historical-Trades)
	* [1.5 Get My Portfolio Matrix](#1.5-Get-My-Portfolio-Matrix)
* [2. Mean Reversion Trading](#2.-Mean-Reversion-Trading)
	* [2.1 Subclass Mean Reversion Trader from Algorithmic Trader](#2.1-Subclass-Mean-Reversion-Trader-from-Algorithmic-Trader)
	* [2.2 Create and Start Mean Reversion Trader](#2.2-Create-and-Start-Mean-Reversion-Trader)
	* [2.3 View Pending Orders](#2.3-View-Pending-Orders)
	* [2.4 View Recent Trades](#2.4-View-Recent-Trades)
* [3. Momentum Trading](#3.-Momentum-Trading)
	* [3.1 Subclass Momentum Trader from Algorithmic Trader](#3.1-Subclass-Momentum-Trader-from-Algorithmic-Trader)
	* [3.2 Create and Start Momentum Trader](#3.2-Create-and-Start-Momentum-Trader)
	* [3.3 View Pending Orders](#3.3-View-Pending-Orders)
	* [3.4 View Recent Trades](#3.4-View-Recent-Trades)
* [4. Pairs Trading](#4.-Pairs-Trading)
	* [4.1 Subclass Pairs Trader from Algorithmic Trader](#4.1-Subclass-Pairs-Trader-from-Algorithmic-Trader)
	* [4.2 Create and Start Pairs Trader](#4.2-Create-and-Start-Pairs-Trader)
	* [4.3 View Pending Orders](#4.3-View-Pending-Orders)
	* [4.4 View Recent Trades](#4.4-View-Recent-Trades)
* [5. Backtesting](#5.-Backtesting)
	* [5.1 Get Symbol Historical Data](#5.1-Get-Symbol-Historical-Data)
	* [5.2 Setup the Mean Reversion Trader](#5.2-Setup-the-Mean-Reversion-Trader)
	* [5.3 Backtest Mean Reversion Trader](#5.3-Backtest-Mean-Reversion-Trader)


# Setup Libraries

In [1]:
# Import packages.
import datetime
import json
import os
import sys
import time

import pandas as pd
import numpy as np
import redis
from pandas.core.common import SettingWithCopyWarning

import warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# Import CloudEx.
import cloud_ex

# Import AlgorithmicTrader helper class.
from algorithmic_trader import AlgorithmicTrader
from algorithmic_trader import summarize_historical_trades_df

# Start Redis backend.
os.system("redis-server --daemonize yes")
time.sleep(1)

# Get CloudEx and VM-specific config 
def get_vm_config():
    with open("vm_config.json", "r") as read_file:
        config = json.load(read_file)
    return config

config = get_vm_config()

# Utility Functions

In [2]:
ORDER_FIELDS_LIST = [
    'Symbol', 'OrderID', 'CancelID', 'ClientID', 'OrderType', 'OrderAction',
    'SubmitTimestamp', 'GatewayTimestamp', 'EnqueueTimestamp',
    'DequeueTimestamp', 'OrderSerialNum', 'LimitPrice', 'ResultType','NumShares'
]

TRADE_FIELDS_LIST = [
    "Symbol", "BuyerSerialNum", "SellerSerialNum", "BuyerOrderID",
    "SellerOrderID", "BuyerClientID", "SellerClientID", "ExecPrice",
    "CashTraded", "SharesTraded", "CreationTimestamp", "ReleaseTimestamp",
    "TradeSerialNum"
]

'''
Takes in a cloud_ex.VectorOrder with serialized orders and returns a DataFrame
'''
def OrderDF(order_vec):
    if not len(order_vec):
        return pd.DataFrame(columns=ORDER_FIELDS_LIST)
    df = pd.DataFrame(order_vec).applymap(lambda x:x.SerializeOrder())[0].str.split('|', expand=True)
    df.columns = ORDER_FIELDS_LIST
    for label in ['SubmitTimestamp', 'GatewayTimestamp', 'EnqueueTimestamp',
                  'DequeueTimestamp', 'OrderSerialNum', 'LimitPrice','NumShares']:
        df.loc[:, label] = pd.to_numeric(df[label], errors='coerce')
    return df

'''
Takes in a cloud_ex.VectorOrder with serialized trades and returns a DataFrame
'''
def TradeDF(trade_vec):
    if not len(trade_vec):
        return pd.DataFrame(columns=TRADE_FIELDS_LIST)
    df = pd.DataFrame(trade_vec).applymap(lambda x:x.SerializeTrade())[0].str.split('|', expand=True)
    df.columns = TRADE_FIELDS_LIST
    for label in ["ExecPrice", "CashTraded", "SharesTraded",
                  "CreationTimestamp", "ReleaseTimestamp", "TradeSerialNum"]:
        df.loc[:, label] = pd.to_numeric(df[label], errors='coerce')
    return df

'''
Takes in a cloud_ex.MapStringOrder mapping Order ID strings to outstanding the coorresponding orders, 
and returns a DataFrame
'''
def OutstandingOrderDF(outstanding_orders):
    if not len(outstanding_orders):
        return pd.DataFrame(columns=ORDER_FIELDS_LIST)
    df = (pd.DataFrame(outstanding_orders.items())[1]).apply(lambda x:x.SerializeOrder()).str.split('|', expand=True)
    df.columns = ORDER_FIELDS_LIST
    for label in ['SubmitTimestamp', 'GatewayTimestamp', 'EnqueueTimestamp',
                  'DequeueTimestamp', 'OrderSerialNum', 'LimitPrice','NumShares']:
        df.loc[:, label] = pd.to_numeric(df[label], errors='coerce')
    return df 

# 1. Instantiate Your CloudEx Trader Object

## 1.1 Create Trader Object 

This object will instantiate a connection to a gateway in CloudEx. This gateway will relay orders to the matching engine and market data to your client.

**Arguments to `cloud_ex.Trader` constructor:**
- gateway_ip `str` - The IP address for the gateway assigned to you
- client_id `str` - Your client identifier
- client_token `str` - Your client token

In [3]:
# Get relevant fields from VM-specific config. Token is yours only, so don't make it public.
gateway_ip = config["gateway_ip"]
client_id = config["client_id"]
client_token = config["client_token"]

# Clear any existing data locally.
redis_api = redis.Redis()
redis_api.flushall();

# Create CloudEx base trader object.
trader = cloud_ex.Trader(gateway_ip, client_id, client_token)

In [4]:
# Get a list of all symbols available for trading.
symbol_list = trader.GetSymbols()
print("These are all the {} symbols available for trading at the CloudEx exchange: {}".format(
    len(symbol_list),symbol_list))

These are all the 100 symbols available for trading at the CloudEx exchange: ['AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'AG', 'AH', 'AI', 'AJ', 'AK', 'AL', 'AM', 'AN', 'AO', 'AP', 'AQ', 'AR', 'AS', 'AT', 'AU', 'AV', 'AW', 'AX', 'AY', 'AZ', 'BA', 'BB', 'BC', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BK', 'BL', 'BM', 'BN', 'BO', 'BP', 'BQ', 'BR', 'BS', 'BT', 'BU', 'BV', 'BW', 'BX', 'BY', 'BZ', 'CA', 'CB', 'CC', 'CD', 'CE', 'CF', 'CG', 'CH', 'CI', 'CJ', 'CK', 'CL', 'CM', 'CN', 'CO', 'CP', 'CQ', 'CR', 'CS', 'CT', 'CU', 'CV', 'CW', 'CX', 'CY', 'CZ', 'DA', 'DB', 'DC', 'DD', 'DE', 'DF', 'DG', 'DH', 'DI', 'DJ', 'DK', 'DL', 'DM', 'DN', 'DO', 'DP', 'DQ', 'DR', 'DS', 'DT', 'DU', 'DV']


## 1.2 Get all My Pending Orders

In [5]:
outstanding_orders = cloud_ex.MapStringOrder()
trader.GetOutstandingOrders(outstanding_orders)

print("You have {} outstanding orders.".format(len(outstanding_orders)))

# Transform outstanding orders into a DataFrame
outstanding_orders = OutstandingOrderDF(outstanding_orders)
outstanding_orders

You have 0 outstanding orders.


,Symbol,OrderID,CancelID,ClientID,OrderType,OrderAction,SubmitTimestamp,GatewayTimestamp,EnqueueTimestamp,DequeueTimestamp,OrderSerialNum,LimitPrice,ResultType,NumShares


## 1.3 Get All My Historical Orders

In [6]:
# Get all my historical orders
my_historical_orders = cloud_ex.VectorOrder()

trader.GetAllHistoricalOrders(my_historical_orders)
print("You have submitted a total of {} order(s).".format(len(my_historical_orders)))
my_historical_orders_df = OrderDF(my_historical_orders)

my_historical_orders_df

You have submitted a total of 0 order(s).


,Symbol,OrderID,CancelID,ClientID,OrderType,OrderAction,SubmitTimestamp,GatewayTimestamp,EnqueueTimestamp,DequeueTimestamp,OrderSerialNum,LimitPrice,ResultType,NumShares


## 1.4 Get All My Historical Trades

In [7]:
# Get all my historical trades
my_historical_trades = cloud_ex.VectorTrade()

trader.GetAllHistoricalTrades(my_historical_trades)
print("You have made a total of {} trade(s).".format(len(my_historical_trades)))
my_historical_trades_df = TradeDF(my_historical_trades)

my_historical_trades_df

You have made a total of 0 trade(s).


,Symbol,BuyerSerialNum,SellerSerialNum,BuyerOrderID,SellerOrderID,BuyerClientID,SellerClientID,ExecPrice,CashTraded,SharesTraded,CreationTimestamp,ReleaseTimestamp,TradeSerialNum


## 1.5 Get My Portfolio Matrix

In [8]:
portfolio_mat = cloud_ex.MapStringInt()

trader.GetPortfolioMatrix(portfolio_mat)

print(portfolio_mat)

MapStringInt{AA: 10000, AB: 10000, AC: 10000, AD: 10000, AE: 10000, AF: 10000, AG: 10000, AH: 10000, AI: 10000, AJ: 10000, AK: 10000, AL: 10000, AM: 10000, AN: 10000, AO: 10000, AP: 10000, AQ: 10000, AR: 10000, AS: 10000, AT: 10000, AU: 10000, AV: 10000, AW: 10000, AX: 10000, AY: 10000, AZ: 10000, BA: 10000, BB: 10000, BC: 10000, BD: 10000, BE: 10000, BF: 10000, BG: 10000, BH: 10000, BI: 10000, BJ: 10000, BK: 10000, BL: 10000, BM: 10000, BN: 10000, BO: 10000, BP: 10000, BQ: 10000, BR: 10000, BS: 10000, BT: 10000, BU: 10000, BV: 10000, BW: 10000, BX: 10000, BY: 10000, BZ: 10000, CA: 10000, CASH: 50000000, CB: 10000, CC: 10000, CD: 10000, CE: 10000, CF: 10000, CG: 10000, CH: 10000, CI: 10000, CJ: 10000, CK: 10000, CL: 10000, CM: 10000, CN: 10000, CO: 10000, CP: 10000, CQ: 10000, CR: 10000, CS: 10000, CT: 10000, CU: 10000, CV: 10000, CW: 10000, CX: 10000, CY: 10000, CZ: 10000, DA: 10000, DB: 10000, DC: 10000, DD: 10000, DE: 10000, DF: 10000, DG: 10000, DH: 10000, DI: 10000, DJ: 10000, DK:

# 2. Mean Reversion Trading

## 2.1 Subclass Mean Reversion Trader from Algorithmic Trader

In [9]:
class meanReversionTrader(AlgorithmicTrader):
    def __init__(self, trader, symbol_list, bin_interval_ms=500):
        """
        Mean Reversion Trader Class.

        :param trader(cloud_ex.Trader): CloudEx's base Trader object.
        :param symbol_list: List of ticker symbols (str) to fetch data for.
        :param bin_interval_ms: Frequency with which to bin trading data in milliseconds (int).
        """
        # Initialize AlgorithmicTrader.
        AlgorithmicTrader.__init__(self, trader, symbol_list, bin_interval_ms=bin_interval_ms)

    def algorithm(self, df, **kwargs):
        """ 
        Calculate buy sell points for an equity.
        A buy is triggered when the moving average is threshold% below the price.
        A sell is triggered when the moving average is threshold% above the price.
        For this implementation we evaluate the stock price at the close of each bin_interval_ms time interval.

        :param df: Dataframes with prices for an equity (pd.DataFrame).
        Mean reversion specific arguments:
          :param ma: Number of bins to consider in the moving average (int).
          :param threshold: Amount ma is below/above price (as a percentage) for which to buy/sell (float).

        Returns: 
            An (action, price) pair.
            
            'action' is None when no action is determined to be taken. This happens either when we do not have enough
            data to build a moving average estimate or if the price is within a small threshold above/below
            the moving average. In this case, 'price' is also None.

            'action' is 'Buy' when the price rises above the threshold defined for the moving average. 
                The buy price is set to the most recent closing price.
                
            'action' is 'Sell' when the price drops below the threshold defined for the moving average. 
                The sell price is set to the most recent closing price.
        """
        # Unpack keyword arguments.
        ma = kwargs['ma']
        threshold = kwargs['threshold']
        
        # Handle case when data not available to build moving average.
        if len(df) < ma:
            return None, None
        
        # Select the last ma rows from our dataframe for processing.
        df = df.iloc[-1*ma:]

        df['MA'] = df['ClosePrice'].rolling(ma).mean()
        row = df.iloc[-1]

        # Check if moving average price is defined.
        # This value will be undefined for the first `ma` steps.
        if np.isnan(row['MA']):
            return None, None

        buy_cutoff = (1.0 - threshold / 100.0) * row['MA']
        sell_cutoff = (1.0 + threshold / 100.0) * row['MA']
        p_s = row['ClosePrice']
        if p_s <= buy_cutoff:
            return 'Buy', p_s
        elif p_s >= sell_cutoff:
            return 'Sell', p_s
        else:
            return None, None

## 2.2 Create and Start Mean Reversion Trader

First, we will create a mean reversion trader.
Once created the trader will pull data from CloudEx, process the data
and lastly place buy or sell orders.

The `meanReversionTrader` object is created with the following parameters:
- trader `cloud_ex.Trader`: The CloudEx base trader object 
- symbol_list `List(str)`: A list of symbols whose market data we would like to subscribe to

We then invoke the trading loop with the following arguments
- symbol `str` - The symbol we would like to trade
- num_shares `int` - Number of shares to buy/sell per order
- num_orders `int` - Number of total orders to place 
- wait_interval `float` - Time to wait between placing orders (s) 
- ma `int` - Number of steps in the moving average window 
- threshold `float` - % above or below the moving average at which we will place orders 

When you have the above information, you can run the order loop using the `meanReversionTrader.trade` function.

In [10]:
# Algorithm Settings.
symbol = 'AA'
num_shares = 100
num_orders = 20
wait_interval = 0.5
moving_average = 29
threshold = 1

# Create and run mean reversion trader.
mrt = meanReversionTrader(trader, [symbol])
mrt_order_id_list = mrt.trade(symbol,
                      num_shares,
                      num_orders,
                      wait_interval,
                      ma=moving_average,
                      threshold=threshold)
num_submitted_orders = len([order_id for order_id in mrt_order_id_list if order_id])
print("\n\nSubmitted orders in {} out of {} iterations".format(num_submitted_orders, len(mrt_order_id_list)))

no new trades
Did not submit an order in this iteration.
no new trades
Did not submit an order in this iteration.
8 new trades
Did not submit an order in this iteration.
no new trades
Did not submit an order in this iteration.
1 new trades
Did not submit an order in this iteration.
no new trades
Did not submit an order in this iteration.
no new trades
Did not submit an order in this iteration.
1 new trades
Did not submit an order in this iteration.
no new trades
Did not submit an order in this iteration.
no new trades
Did not submit an order in this iteration.
1 new trades


Placing trade, symbol= AA  price= 31.0  submit_num_shares= 100  buy= True


no new trades


Placing trade, symbol= AA  price= 31.0  submit_num_shares= 100  buy= True


no new trades


...

no new trades


Placing trade, symbol= AA  price= 29.0  submit_num_shares= 100  buy= True


1 new trades


Placing trade, symbol= AA  price= 29.0  submit_num_shares= 100  buy= True




Submitted orders in 10 out of 20 iterations


## 2.3 View Pending Orders

You can view all outstanding orders (those that have not been fulfilled) by calling the `trader.GetOutstandingOrders` function. 

Doing so can give you the information needed to cancel any or all orders.


In [11]:
outstanding_orders = cloud_ex.MapStringOrder()
mrt.trader.GetOutstandingOrders(outstanding_orders)

print("You have {} outstanding orders.".format(len(outstanding_orders)))

# Transform outstanding orders into a DataFrame
outstanding_orders = OutstandingOrderDF(outstanding_orders)
outstanding_orders

You have 5 outstanding orders.


,Symbol,OrderID,CancelID,ClientID,OrderType,OrderAction,SubmitTimestamp,GatewayTimestamp,EnqueueTimestamp,DequeueTimestamp,OrderSerialNum,LimitPrice,ResultType,NumShares
0,AA,AA.0999711602867132153943600000516,NULL,C16,L,B,1602867132153677,1602867132153954,1602867132154141,1602867132154304,6,29,V,98
1,AA,AA.0999711602867132659255600000616,NULL,C16,L,B,1602867132659036,1602867132659267,1602867132659458,1602867132659618,7,29,V,100
2,AA,AA.0999711602867133208178600000716,NULL,C16,L,B,1602867133207918,1602867133208192,1602867133208409,1602867133208542,8,29,V,100
3,AA,AA.0999711602867133714495600000816,NULL,C16,L,B,1602867133714269,1602867133714509,1602867133714743,1602867133714859,9,29,V,100
4,AA,AA.0999711602867134264463600000916,NULL,C16,L,B,1602867134264259,1602867134264476,1602867134264707,1602867134264827,10,29,V,100


## 2.4 View Recent Trades

For any of the symbols we have subscribed to, we can pull recent trades. We will use the `trader.GetRecentTrades` to do so.
- symbol `str` - Ticker we will fetch data for.
- trade_vec `cloud_ex.VectorTrade` - Datastructure to hold the recent trades. 
- start_fetch_time `int` - Time (in microseconds) to start fetching data from.


In [12]:
# 'symbol' should be same symbol we used before
symbol = 'AA'
past_seconds = 60
trade_vec = cloud_ex.VectorTrade()
start_fetch_time_us = int((time.time() - past_seconds) * 1e6) # We will fetch from past_seconds seconds ago to now.

success = mrt.trader.GetRecentTrades(symbol, trade_vec, start_fetch_time_us)
if not success:
    print("Error getting {symbol} trades. Check that {symbol} is in your active symbol list".format(symbol=symbol))
else:
    print("There were {} Trades for {} in the last {} seconds.".format(len(trade_vec), symbol, past_seconds))

# Transform recent trades into a DataFrame
recent_trade_df = TradeDF(trade_vec)
recent_trade_df

There were 20 Trades for AA in the last 60 seconds.


,Symbol,BuyerSerialNum,SellerSerialNum,BuyerOrderID,SellerOrderID,BuyerClientID,SellerClientID,ExecPrice,CashTraded,SharesTraded,CreationTimestamp,ReleaseTimestamp,TradeSerialNum
0,AA,0,0,NULL,NULL,NULL,NULL,29,29,1,1602867134124948,1602867134125298,100394
1,AA,0,0,NULL,NULL,NULL,NULL,29,29,1,1602867132674937,1602867132675287,100295
2,AA,0,0,NULL,NULL,NULL,NULL,29,29,1,1602867131653969,1602867131654319,100217
3,AA,0,0,NULL,NULL,NULL,NULL,31,3100,100,1602867131605635,1602867131605985,100213
4,AA,0,0,NULL,NULL,NULL,NULL,31,3100,100,1602867131056028,1602867131056378,100179
5,AA,0,0,NULL,NULL,NULL,NULL,31,3100,100,1602867130507071,1602867130507421,100140
6,AA,0,0,NULL,NULL,NULL,NULL,30,30,1,1602867130377955,1602867130378305,100131
7,AA,0,0,NULL,NULL,NULL,NULL,31,3100,100,1602867130375241,1602867130375591,100130
8,AA,0,0,NULL,NULL,NULL,NULL,31,3100,100,1602867130375238,1602867130375588,100129
9,AA,0,0,NULL,NULL,NULL,NULL,31,31,1,1602867129154697,1602867129155047,100044


# 3. Momentum Trading

## 3.1 Subclass Momentum Trader from Algorithmic Trader

In [13]:
class momentumTrader(AlgorithmicTrader):
    def __init__(self, trader, symbol_list, bin_interval_ms=500):
        """
        Momentum Trader Class.

        :param trader(cloud_ex.Trader): CloudEx's base Trader object
        :param symbol_list: List of ticker symbols (str) to fetch data for.
        :param bin_interval_ms: Frequency with which to bin trading data in milliseconds (int).
        """
        # Initialize Trader.
        AlgorithmicTrader.__init__(self, trader, symbol_list, bin_interval_ms=bin_interval_ms)

    def algorithm(self, df, **kwargs):
        """
        Calculate buy sell points for an equity.
        A buy is triggered when the weighted rate of change for the past two time intervals is above a given threshold.
        A sell is triggered when the weighted rate of change for the past two time intervals is below a given threshold.
        For this implementation we evaluate the rate of change for stock price from open to close for each bin_interval_ms time interval.

        :param df: Dataframes with prices for an equity (pd.DataFrame).
        Momentum specific arguments:
        :param threshold: Weighted rate of change above/below which we will buy/sell (float).
        :param p1: weight for t-1 rate of change, Note: p1+p2=1 (float).
        :param p2: Weight for t-2 rate of change, Note: p1+p2=1 (float).

        Returns: 
            An (action, price) pair.
            
            'action' is None when no action is determined to be taken. This happens either when we do not have enough
            data to build a moving average estimate or if the price is within a small threshold above/below
            the moving average. In this case, 'price' is also None.

            'action' is 'Buy' when the price rises above the threshold defined for the moving average. 
                The buy price is set to the most recent closing price.
                
            'action' is 'Sell' when the price drops below the threshold defined for the moving average. 
                The sell price is set to the most recent closing price.
        """
        # Handle case when data not available to get momentum.
        if len(df) < 2:
            return None, None

        # Select the last few rows from our dataframe for processing.
        df = df.iloc[-2:]
        
        # Unpack keyword arguments
        threshold = kwargs['threshold']
        p1 = kwargs['p1']
        p2 = kwargs['p2']

        # Calculate the rate of change for our equity at each timestep.
        df['Perc Change'] = (df['ClosePrice'] / df['OpenPrice'] - 1)
        row = df.iloc[-1]
        prev_row = df.iloc[-2]

        # Check if percentage change of previous row is defined.
        # This value will be undefined for the first step.
        if np.isnan(prev_row['Perc Change']):
            return None, None

        # Find the weighted average of recent moves (as a percent of stock price).
        weighted_ave_roc = p1 * row['Perc Change'] + p2 * prev_row[
            'Perc Change']
        print("Momentum value= ", weighted_ave_roc*100)
        p_s = row['ClosePrice']
        
        if weighted_ave_roc >= threshold / 100.0:
            return 'Buy', p_s
        elif weighted_ave_roc <= -threshold / 100.0:
            return 'Sell', p_s
        else:
            return None, None


## 3.2 Create and Start Momentum Trader

First, we will create a momentum trader.
Once created the trader will pull data from CloudEx, process the data
and lastly place buy or sell orders.

The `momentumTrader` object is created with the following parameters:
- trader `cloud_ex.Trader`: The CloudEx base trader object 
- symbol_list `List(str)`: A list of symbols whose market data we would like to subscribe to

We then invoke the trading loop with the following arguments
- symbol `str` - The symbol we would like to trade 
- num_shares `int` - Number of shares to buy/sell per order  
- num_orders `int` - Number of total orders to place 
- wait_interval `float` - Time to wait between placing orders (s) 
- threshold `float` - % above or below the moving average at which we will place orders 
- p1 `float` - Weight for t-1 rate of change 
- p2 `float` - Weight for t-2 rate of change 

When you have the above information, you can run the order loop using the `momentumTrader.trade` function.

In [ ]:
symbol = 'AI'
num_shares = 100
num_orders = 10
wait_interval = 3
threshold = 1
p1 = .5
p2 = .5

# Create and run momentum trader.
mt = momentumTrader(trader, [symbol], bin_interval_ms=wait_interval*1000)
mt_order_id_list = mt.trade(symbol,
                      num_shares,
                      num_orders,
                      wait_interval,
                      p1=p1,
                      p2=p2,
                      threshold=threshold)

num_submitted_orders = len([order_id for order_id in mt_order_id_list if order_id])
print("\n\nSubmitted orders in {} out of {} iterations".format(num_submitted_orders, len(mt_order_id_list)))

no new trades
Did not submit an order in this iteration.
17 new trades
Momentum value=  -0.8771929824561431
Did not submit an order in this iteration.
2 new trades
Momentum value=  -1.785714285714285


Placing trade, symbol= AI  price= 54.0  submit_num_shares= 100  buy= False


## 3.3 View Pending Orders

As before, you can view all outstanding orders (those that have not been fulfilled) by calling the `trader.GetOutstandingOrders` function. 

Doing so can give you the information needed to cancel any or all orders.


In [ ]:
outstanding_orders = cloud_ex.MapStringOrder()
mt.trader.GetOutstandingOrders(outstanding_orders)

print("You have {} outstanding orders.".format(len(outstanding_orders)))

# Transform outstanding orders into a DataFrame
outstanding_orders = OutstandingOrderDF(outstanding_orders)
outstanding_orders

## 3.4 View Recent Trades

As before, for any of the symbols we have subscribed to, we can pull recent trades. We will use the `trader.GetRecentTrades` to do so.
- symbol `str` - Ticker we will fetch data for.
- trade_vec `cloud_ex.VectorTrade` - Datastructure to hold the recent trades. 
- start_fetch_time `int` - Time (in microseconds) to start fetching data from.


In [ ]:
# 'symbol' should be same symbol we used before
symbol = 'AI'
past_seconds = 60
trade_vec = cloud_ex.VectorTrade()
start_fetch_time_us = int((time.time() - past_seconds) * 1e6) # We will fetch from past_seconds seconds ago to now.

success = mt.trader.GetRecentTrades(symbol, trade_vec, start_fetch_time_us)
if not success:
    print("Error getting {symbol} trades. Check that {symbol} is in your active symbol list".format(symbol=symbol))
else:
    print("There were {} Trades for {} in the last {} seconds.".format(len(trade_vec), symbol, past_seconds))

# Transform recent trades into a DataFrame
recent_trade_df = TradeDF(trade_vec)
recent_trade_df

# 4. Pairs Trading

## 4.1 Subclass Pairs Trader from Algorithmic Trader

In [ ]:
class pairsTrader(AlgorithmicTrader):
    def __init__(self, trader, symbol_list, bin_interval_ms=500):
        """
        Pairs Trader Class.

        :param trader(cloud_ex.Trader): CloudEx's base Trader object
        :param symbol_list: List of ticker symbols (str) to fetch data for.
        :param bin_interval_ms: Frequency with which to bin trading data in milliseconds (int).
        """
        # Initialize Trader.
        AlgorithmicTrader.__init__(self, trader, symbol_list, bin_interval_ms=bin_interval_ms)

    def algorithm(self, df1, symbol2, ma, threshold):
        """ Calculate buy sell points for an equity.
            We will difference the two time series in our pairs trading strategy in the following order:
                differenced_time_series(t) = symbol_1(t) - symbol_2(t)
            A buy is triggered when the moving average (of the difference time series) is threshold% below the current price.
            A sell is triggered when the moving average (of the difference time series) is threshold% above the current price.
            For this implementation we evaluate the stock price at the close of each bin_interval_ms time interval.
            
        Arguments:
            :param df1: Dataframes with prices for the first symbol (pd.DataFrame).
            :param symbol2: the second symbol in our pairs trading strategy (str).
            :param ma: Number of periods to consider in the moving average (int).
            :param threshold: Amount ma is below/above price (as a percentage) for which to buy/sell (float).

        Returns: 
            An (action, price) pair.
            
            'action' is None when no action is determined to be taken. This happens either when we do not have enough
            data to build a moving average estimate or if the price is within a small threshold above/below
            the moving average. In this case, 'price' is also None.

            'action' is 'Buy' when the price rises above the threshold defined for the moving average. 
                The buy price is set to the most recent closing price.
                
            'action' is 'Sell' when the price drops below the threshold defined for the moving average. 
                The sell price is set to the most recent closing price.
        """
        # Get dataframe for second symbol.
        self._update_time_and_sales(symbol2)
        # df2 {pd.DataFrame} -- Dataframe with prices for the second equity.
        df2 = self.summarized_time_and_sales_dict[symbol2]
        
        # Handle case when data not available to build moving average.
        if len(df1) < ma or len(df2) < ma:
            return None, None
        
        df1 = df1.iloc[-1*ma:]
        df2 = df2.iloc[-1*ma:]
        
        # Check that the dataframe indexes (timestamps) match for the two symbols we are trading.
        if df1.index.values[-1] != df2.index.values[-1]:
            return None, None
        
        # Get difference time series, pair_df.
        pair_df = df1 - df2
        pair_df['MA'] = pair_df['ClosePrice'].rolling(ma).mean()
        row = pair_df.iloc[-1]

        # Check if moving average price is defined.
        # This value will be undefined for the first `ma` steps.
        if np.isnan(row['MA']):
            return None, None

        buy_cutoff = (1.0 - threshold / 100.0) * row['MA']
        sell_cutoff = (1.0 + threshold / 100.0) * row['MA']
        p_s = row['ClosePrice']

        p_s1 = df1.iloc[-1]['ClosePrice']
        if p_s <= buy_cutoff:
            return 'Buy', p_s1  # Sell s1 (can also buy s2)
        elif p_s >= sell_cutoff:
            return 'Sell', p_s1  # Buy s1 (can also sell s2)
        else:
            return None, None

## 4.2 Create and Start Pairs Trader

First, we will create a pairs trader.
Once created the trader will pull data from CloudEx, process the data
and lastly place buy or sell orders.

The `pairsTrader` object is created with the following parameters:
- trader `cloud_ex.Trader`: The CloudEx base trader object 
- symbol_list `List(str)`: A list of symbols whose market data we would like to subscribe to

We then invoke the trading loop with the following arguments
- symbol1, symbol2 `str` - The two symbol we would like to trade using pairs trading
- num_shares `int` - Number of shares to buy/sell per order  
- num_orders `int` - Number of total orders to place 
- wait_interval `float` - Time to wait between placing orders (s) 
- ma `int` - Number of steps in the moving average window 
- threshold `float` - % above or below the moving average at which we will place orders 

When you have the above information, you can run the order loop using the `pairsTrader.trade` function.

In [ ]:
# Algorithm Settings
symbol1 = 'AE'
symbol2 = 'AF'
num_shares = 100
num_orders = 20
wait_interval = .5
moving_average = 10
threshold = 5

# Create and run pairs trader.
pt = pairsTrader(trader, [symbol1, symbol2])
pt_order_id_list = pt.trade(symbol1,
                    num_shares,
                    num_orders,
                    wait_interval,
                    ma=moving_average,
                    threshold=threshold,
                    symbol2=symbol2)

num_submitted_orders = len([order_id for order_id in pt_order_id_list if order_id])
print("\n\nSubmitted orders in {} out of {} iterations".format(num_submitted_orders, len(pt_order_id_list)))

## 4.3 View Pending Orders

As before, you can view all outstanding orders (those that have not been fulfilled) by calling the `trader.GetOutstandingOrders` function. 

Doing so can give you the information needed to cancel any or all orders.

In [ ]:
outstanding_orders = cloud_ex.MapStringOrder()
pt.trader.GetOutstandingOrders(outstanding_orders)

print("You have {} outstanding orders.".format(len(outstanding_orders)))

# Transform outstanding orders into a DataFrame
outstanding_orders = OutstandingOrderDF(outstanding_orders)
outstanding_orders

## 4.4 View Recent Trades

As before, for any of the symbols we have subscribed to, we can pull recent trades. We will use the `trader.GetRecentTrades` to do so.
- symbol `str` - Ticker we will fetch data for.
- trade_vec `cloud_ex.VectorTrade` - Datastructure to hold the recent trades. 
- start_fetch_time `int` - Time (in microseconds) to start fetching data from.

In [ ]:
symbol = 'AE' # Same symbol we used before
past_seconds = 60
trade_vec = cloud_ex.VectorTrade()
start_fetch_time_us = int((time.time() - past_seconds) * 1e6) # We will fetch from past_seconds seconds ago to now.

success = pt.trader.GetRecentTrades(symbol, trade_vec, start_fetch_time_us)
if not success:
    print("Error getting {symbol} trades. Check that {symbol} is in your active symbol list".format(symbol=symbol))
else:
    print("There were {} Trades for {} in the last {} seconds.".format(len(trade_vec), symbol, past_seconds))

# Transform recent trades into a DataFrame
recent_trade_df = TradeDF(trade_vec)
recent_trade_df

# 5. Backtesting

## 5.1 Get Symbol Historical Data

In [ ]:
symbol = 'AA'
seconds_in_past = 60*60*24 # One day
end_time_ms = int(time.time()*1e3)
start_time_ms = end_time_ms - int(seconds_in_past*1e3)

symbol_trades_vec = cloud_ex.VectorTrade()

cloud_ex.MarketDataAPI.PullTrades(config['project_id'], config['bigtable_id'], 
                                          config['table_name'], symbol, start_time_ms, 
                                          end_time_ms, symbol_trades_vec)
print("There are a total of {} trade(s) in the last day.".format(len(symbol_trades_vec)))
symbol_historical_trades_df = TradeDF(symbol_trades_vec)

symbol_historical_trades_df = symbol_historical_trades_df.sort_values(by="CreationTimestamp")
symbol_historical_trades_df

In [ ]:
bin_interval_ms = 500
summarize_historical_trades_df(symbol_historical_trades_df, bin_interval_ms)

## 5.2 Setup the Mean Reversion Trader

In [ ]:
symbol = 'AA'
# Create mean reversion trader for backtesting.
mrt = meanReversionTrader(trader, [symbol])

## 5.3 Backtest Mean Reversion Trader

We can backtest the choice of parameters (`moving_average` and `threshold`) that we have selected by evaluating performance on historical data. 

`backtest()` Arguments:
- symbol_historical_trades_df `pd.DataFrame` - Stock time series dataframe to backtest on. 
- num_shares `int`: Number of shares to buy or sell. 
- init_capital `int`: Amount of capital to start with. 
- init_shares `int`: Number of shares to start with. 
- Specific to mean reversion algorithm:
 - ma `int` - Number of steps in the moving average window 
 - threshold `float` - % above or below the moving average at which we will place orders 

`backtest()` returns:
- ROI (%)
- a list of (time index, action, price) tuples

In [ ]:
# Initial capital and shares we will use for the backtest. 
# Bad parameters.
num_shares = 10
moving_average = 10
threshold = 5
init_capital = 100000
init_shares = 1000

roi, action_list = mrt.backtest(symbol_historical_trades_df,
             num_shares, 
             init_capital, 
             init_shares, 
             ma=moving_average, 
             threshold=threshold)

print(action_list)
print("Mean Reversion Trader, ma={ma} threshold={threshold} ----- ROI={roi}%".format(
    ma=moving_average, threshold=threshold, roi=roi))

In [ ]:
# Initial capital and shares we will use for the backtest. 
# Good parameters.
num_shares = 10
moving_average = 29
threshold = 1
init_capital = 100000
init_shares = 1000

roi, action_list = mrt.backtest(symbol_historical_trades_df,
             num_shares, 
             init_capital, 
             init_shares, 
             ma=moving_average, 
             threshold=threshold)

print(action_list)
print("Mean Reversion Trader, ma={ma} threshold={threshold} ----- ROI={roi}%".format(
    ma=moving_average, threshold=threshold, roi=roi))